In [1]:
from main import generate_estimation
from math import ceil
import time
import json
from datetime import datetime
import threading
import os
# _, numOfCars = generate_estimation()
# print(_)
# numOfCars
# greenTime = min(numOfCars * 2 + 3, 120)
# print(numOfCars, greenTime)

In [2]:
# write code to remove the content of intersection_A.json, intersection_B.json, intersection_C.json, intersection_D.json
# after the simulation is done
os.remove("intersection_A.json")
os.remove("intersection_B.json")
os.remove("intersection_C.json")
os.remove("intersection_D.json")


In [3]:
class TrafficLight:
    def __init__(self, trafficID, lanes):
        self.trafficID = trafficID
        self.lanes = lanes
        self.totalCars = 0
        self.carsPerLane = 0
        self.greenTime = 3

    def update_traffic(self, totalCars):
        self.totalCars += totalCars
        self.carsPerLane = ceil(totalCars / self.lanes)
        # self.greenTime = min(self.carsPerLane * 2 + 3, 50)
        self.greenTime = max(min(ceil((self.totalCars * 2.5)/(self.lanes +1)), 50),3)
    def getTotalCars(self):
        return self.totalCars
    def getGreenTime(self):
        return self.greenTime
    def getTrafficID(self):
        return self.trafficID
    def resetGreenTime(self):
        self.greenTime = 3
    def resetTotalCars(self):
        if self.greenTime > 3:
            self.totalCars -= max(0, (self.greenTime*(self.lanes + 1))//2.5) 
        else:
            self.totalCars = 0
    def __str__(self):
        return (f"Traffic Light {self.trafficID} -> Lanes: {self.lanes}, Total Cars: {self.totalCars}, "
                f"Cars per Lane: {self.carsPerLane}, Green Time: {self.greenTime}s")
       # totalCars = 20  # Simulated number of cars

class Intersection:
    def __init__(self, intersectionID, trafficLights):
        self.intersectionID = intersectionID
        self.trafficLights = trafficLights
    def getIntersectionID(self):
        return self.intersectionID
    def getTrafficLights(self):
        return self.trafficLights
    def getSignalOn(self):
        for light in self.trafficLights:
            if light.greenTime > 0:
                return light.getTrafficID
        return None
    def getTrafficLightwithID(self, trafficID):
        for light in self.trafficLights:
            if light.getTrafficID() == trafficID:
                return light
        return None


class CircularTrafficLights:
    def __init__(self, intersection):
        self.lights = intersection.getTrafficLights()
        self.intersection = intersection
        self.index = 0

    def __iter__(self):
        return self

    def __next__(self):
        if not self.lights:
            raise StopIteration
        light = self.lights[self.index]
        self.index = (self.index + 1) % len(self.lights)
        return light
    def getIntersection(self):
        return self.intersection


    # def update_light(self, trafficID, totalCars, carsPerLane, greenTime):
    #     for light in self.lights:
    #         if light.trafficID == trafficID:
    #             light.update_traffic(totalCars, carsPerLane, greenTime)
    #             print(f"Updated Traffic Light {trafficID}")

def save_to_database(data, filename='testA.json'):
    try:
        with open(filename, 'r') as json_file:
            existing_data = json.load(json_file)
            if not isinstance(existing_data, list):
                existing_data = []
    except (FileNotFoundError, json.JSONDecodeError):
        existing_data = []

    
    data_dict = {
        f"Cycle {data[0]}": {
            "trafficID": data[1],
            "num_cars": data[2],
            "weatherStamp:": data[3],
            "timeStamp": data[4],
            "dateStamp": data[5],
            "greenTime": data[6]
        }
    }

    existing_data.append(data_dict)  # Use append to add the dictionary to the list

    with open(filename, 'w') as json_file:
        json.dump(existing_data, json_file, indent=4)

In [4]:


intersection1 = [
    TrafficLight(1, 4), # trafficID, lanes
    TrafficLight(2, 4),
    TrafficLight(3, 4),
    TrafficLight(4, 4)
]
intersection2 = [
    TrafficLight(1, 4), # trafficID, lanes
    TrafficLight(2, 4),
    TrafficLight(3, 4),
    TrafficLight(4, 4)
]
intersection3 = [
    TrafficLight(1, 4), # trafficID, lanes
    TrafficLight(2, 4),
    TrafficLight(3, 4),
    TrafficLight(4, 4)
]
intersection4 = [
    TrafficLight(1, 4), # trafficID, lanes
    TrafficLight(2, 4),
    TrafficLight(3, 4),
    TrafficLight(4, 4)
]
network = [Intersection('A',intersection1), Intersection('B',intersection2), Intersection('C',intersection3), Intersection('D',intersection4)]
# traffic_circle = []
# for i, intersection in enumerate(network):
#     traffic_circle.append(CircularTrafficLights(intersection))

traffic_circle1 = CircularTrafficLights(network[0])
traffic_circle2 = CircularTrafficLights(network[1])
traffic_circle3 = CircularTrafficLights(network[2])
traffic_circle4 = CircularTrafficLights(network[3])

In [5]:
def setup_delayed_task(func, delay, *args):
    timer = threading.Timer(delay, func, args=args)
    timer.start()
    return
def procesIntersectionWithThermal(circularTrafficLights):
    i = 0
    cycle = 0
    delay_seconds = 30 
    for light in circularTrafficLights:
        if(i % len(circularTrafficLights.getIntersection().getTrafficLights()) == 0):
            cycle += 1
        if(cycle == 11):
            break
        i += 1
        time.sleep(2)
        while True:
            try:
                history, numOfCars = generate_estimation()
                break  # Exit the loop if the function succeeds
            except Exception as e:
                print(f"Error occurred: {e}. Retrying...")

        light.update_traffic(numOfCars)
        weatherStamp = history[0][2]
        timeStamp = datetime.now().strftime("%H:%M:%S")
        dateStamp = datetime.now().strftime("%Y-%m-%d")

        data = [
            cycle, light.trafficID, numOfCars,
            weatherStamp, timeStamp, dateStamp,
            light.getGreenTime()
        ]
        save_to_database(data, f"intersection_{circularTrafficLights.getIntersection().getIntersectionID()}.json")
        print(light)
        if circularTrafficLights.getIntersection().getIntersectionID() == "A":
            setup_delayed_task(L1, delay_seconds, circularTrafficLights.getIntersection(), light.getTrafficID(), light.getTotalCars())
            setup_delayed_task(L2, delay_seconds, circularTrafficLights.getIntersection(), light.getTrafficID(), light.getTotalCars())
        elif circularTrafficLights.getIntersection().getIntersectionID() == "B":
            setup_delayed_task(L1, delay_seconds, circularTrafficLights.getIntersection(), light.getTrafficID(), light.getTotalCars())
            setup_delayed_task(L4, delay_seconds, circularTrafficLights.getIntersection(), light.getTrafficID(), light.getTotalCars())
        elif circularTrafficLights.getIntersection().getIntersectionID() == "C":
            setup_delayed_task(L2, delay_seconds, circularTrafficLights.getIntersection(), light.getTrafficID(), light.getTotalCars())
            setup_delayed_task(L3, delay_seconds, circularTrafficLights.getIntersection(), light.getTrafficID(), light.getTotalCars())
        elif circularTrafficLights.getIntersection().getIntersectionID() == "D":
            setup_delayed_task(L3, delay_seconds, circularTrafficLights.getIntersection(), light.getTrafficID(), light.getTotalCars())
            setup_delayed_task(L4, delay_seconds, circularTrafficLights.getIntersection(), light.getTrafficID(), light.getTotalCars())
        time.sleep(light.getGreenTime())
        light.resetTotalCars()
        light.resetGreenTime()


def processIntersectionWithSensors(circularTrafficLights):

    i = 0
    cycle = 0
    delay_seconds = 30
    for light in circularTrafficLights:
        if(i % len(circularTrafficLights.getIntersection().getTrafficLights()) == 0):
            cycle += 1
        if(cycle == 11):
            break
        i += 1
        time.sleep(2)
        
        weatherStamp = None
        timeStamp = datetime.now().strftime("%H:%M:%S")
        dateStamp = datetime.now().strftime("%Y-%m-%d")
        numOfCars = light.getTotalCars()
        data = [
            cycle, light.trafficID, numOfCars,
            weatherStamp, timeStamp, dateStamp,
            light.getGreenTime()
        ]
        save_to_database(data, f"intersection_{circularTrafficLights.getIntersection().getIntersectionID()}.json")
        if circularTrafficLights.getIntersection().getIntersectionID() == "A":
            setup_delayed_task(L1, delay_seconds, circularTrafficLights.getIntersection(),light.getTrafficID(), light.getTotalCars())
            setup_delayed_task(L2, delay_seconds, circularTrafficLights.getIntersection(),light.getTrafficID(), light.getTotalCars())
        elif circularTrafficLights.getIntersection().getIntersectionID() == "B":
            setup_delayed_task(L1, delay_seconds, circularTrafficLights.getIntersection(),light.getTrafficID(), light.getTotalCars())
            setup_delayed_task(L4, delay_seconds, circularTrafficLights.getIntersection(),light.getTrafficID(), light.getTotalCars())
        elif circularTrafficLights.getIntersection().getIntersectionID() == "C":
            setup_delayed_task(L2, delay_seconds, circularTrafficLights.getIntersection(),light.getTrafficID(), light.getTotalCars())
            setup_delayed_task(L3, delay_seconds, circularTrafficLights.getIntersection(),light.getTrafficID(), light.getTotalCars())
        elif circularTrafficLights.getIntersection().getIntersectionID() == "D":
            setup_delayed_task(L3, delay_seconds, circularTrafficLights.getIntersection(),light.getTrafficID(), light.getTotalCars())
            setup_delayed_task(L4, delay_seconds, circularTrafficLights.getIntersection(),light.getTrafficID(), light.getTotalCars())
        time.sleep(light.getGreenTime())
        light.resetTotalCars()
        light.resetGreenTime()
def L1(intersection,trafficLightID, totalCars):
    if(intersection.getIntersectionID() == "A"):
        if(trafficLightID == 1):
            network[1].getTrafficLightwithID(2).update_traffic(int(0.2 * totalCars))
        elif(trafficLightID == 2):
            network[1].getTrafficLightwithID(2).update_traffic(int(0.8 * totalCars))
        elif(trafficLightID == 3):
            network[1].getTrafficLightwithID(2).update_traffic(int(0.2 * totalCars))
    elif(intersection.getIntersectionID() == "B"):
        if(trafficLightID == 1):
            network[0].getTrafficLightwithID(4).update_traffic(int(0.2 * totalCars))
        elif(trafficLightID == 4):
            network[0].getTrafficLightwithID(4).update_traffic(int(0.8 * totalCars))
        elif(trafficLightID == 3):
            network[0].getTrafficLightwithID(4).update_traffic(int(0.2 * totalCars))

def L2(intersection,trafficLightID, totalCars):
    if(intersection.getIntersectionID() == "A"):
        if(trafficLightID == 1):
            network[2].getTrafficLightwithID(1).update_traffic(int(0.8 * totalCars))
        elif(trafficLightID == 2):
            network[2].getTrafficLightwithID(1).update_traffic(int(0.2 * totalCars))
        elif(trafficLightID == 4):
            network[2].getTrafficLightwithID(1).update_traffic(int(0.2 * totalCars))
    elif(intersection.getIntersectionID() == "C"):
        if(trafficLightID == 2):
            network[0].getTrafficLightwithID(3).update_traffic(int(0.2 * totalCars))
        elif(trafficLightID == 4):
            network[0].getTrafficLightwithID(3).update_traffic(int(0.2 * totalCars))
        elif(trafficLightID == 3):
            network[0].getTrafficLightwithID(3).update_traffic(int(0.8 * totalCars))

def L3(intersection,trafficLightID, totalCars):
    if(intersection.getIntersectionID() == "C"):
        if(trafficLightID == 1):
            network[3].getTrafficLightwithID(2).update_traffic(int(0.2 * totalCars))
        elif(trafficLightID == 2):
            network[3].getTrafficLightwithID(2).update_traffic(int(0.8 * totalCars))
        elif(trafficLightID == 3):
            network[3].getTrafficLightwithID(2).update_traffic(int(0.2 * totalCars))
    elif(intersection.getIntersectionID() == "D"):
        if(trafficLightID == 1):
            network[2].getTrafficLightwithID(4).update_traffic(int(0.2 * totalCars))
        elif(trafficLightID == 4):
            network[2].getTrafficLightwithID(4).update_traffic(int(0.8 * totalCars))
        elif(trafficLightID == 3):
            network[2].getTrafficLightwithID(4).update_traffic(int(0.2 * totalCars))

def L4(intersection,trafficLightID, totalCars):
    if(intersection.getIntersectionID() == "B"):
        if(trafficLightID == 2):
            network[3].getTrafficLightwithID(1).update_traffic(int(0.2 * totalCars))
        elif(trafficLightID == 1):
            network[3].getTrafficLightwithID(1).update_traffic(int(0.8 * totalCars))
        elif(trafficLightID == 4):
            network[3].getTrafficLightwithID(1).update_traffic(int(0.2 * totalCars))
    elif(intersection.getIntersectionID() == "D"):
        if(trafficLightID == 2):
            network[1].getTrafficLightwithID(3).update_traffic(int(0.2 * totalCars))
        elif(trafficLightID == 3):
            network[1].getTrafficLightwithID(3).update_traffic(int(0.8 * totalCars))
        elif(trafficLightID == 4):
            network[1].getTrafficLightwithID(3).update_traffic(int(0.2 * totalCars))

In [6]:
threadA = threading.Thread(target=procesIntersectionWithThermal, args=(traffic_circle1,))
threadB = threading.Thread(target=processIntersectionWithSensors, args=(traffic_circle2,))
threadC = threading.Thread(target=processIntersectionWithSensors, args=(traffic_circle3,))
threadD = threading.Thread(target=processIntersectionWithSensors, args=(traffic_circle4,))

threadA.start()
threadB.start()
threadC.start()
threadD.start()

threadA.join()
threadB.join()
threadC.join()
threadD.join()


Traffic Light 1 -> Lanes: 4, Total Cars: 95, Cars per Lane: 24, Green Time: 48s
Traffic Light 2 -> Lanes: 4, Total Cars: 1, Cars per Lane: 1, Green Time: 3s
Traffic Light 3 -> Lanes: 4, Total Cars: 106, Cars per Lane: 27, Green Time: 50s
Traffic Light 4 -> Lanes: 4, Total Cars: 61, Cars per Lane: 16, Green Time: 31s
Error occurred: 'NoneType' object is not subscriptable. Retrying...
Traffic Light 1 -> Lanes: 4, Total Cars: 30.0, Cars per Lane: 8, Green Time: 15s
Traffic Light 2 -> Lanes: 4, Total Cars: 44, Cars per Lane: 11, Green Time: 22s
Traffic Light 3 -> Lanes: 4, Total Cars: 111.0, Cars per Lane: 27, Green Time: 50s
Traffic Light 4 -> Lanes: 4, Total Cars: 94.0, Cars per Lane: 24, Green Time: 47s
Traffic Light 1 -> Lanes: 4, Total Cars: 72.0, Cars per Lane: 18, Green Time: 36s
Traffic Light 2 -> Lanes: 4, Total Cars: 111.0, Cars per Lane: 28, Green Time: 50s
Error occurred: 'NoneType' object is not subscriptable. Retrying...
Traffic Light 3 -> Lanes: 4, Total Cars: 127.0, Cars pe

In [7]:
# cnt = 0
# for light in traffic_circle1:
#     print(light.)
#     cnt+=1
#     if cnt == 4:
#         break

In [8]:
import threading
import time

def function1(arg):
    for i in range(5):
        print(f"Thread {arg} iteration {i}")
        # time.sleep(1)

# Create threads targeting the same function but with different arguments
thread1 = threading.Thread(target=function1, args=("1",))
thread2 = threading.Thread(target=function1, args=("2",))

# Start threads
thread1.start()
thread2.start()

# Wait for both threads to complete
thread1.join()
thread2.join()

print("Both threads have completed their execution")


Thread 1 iteration 0
Thread 1 iteration 1
Thread 1 iteration 2
Thread 1 iteration 3
Thread 1 iteration 4
Thread 2 iteration 0
Thread 2 iteration 1
Thread 2 iteration 2
Thread 2 iteration 3
Thread 2 iteration 4
Both threads have completed their execution
